In [3]:
import qiskit.tools.jupyter
%qiskit_version_table

In [4]:
import numpy as np
from numpy import linalg as LA
from scipy.linalg import expm, sinm, cosm
from numpy import count_nonzero
import math, cmath
from scipy.optimize import fmin, minimize, rosen, rosen_der
from itertools import product, combinations
from copy import copy
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.interpolate import make_lsq_spline, BSpline
from scipy.interpolate import make_interp_spline
from scipy.interpolate import Rbf, InterpolatedUnivariateSpline
from scipy.interpolate import interp1d
from scipy.sparse import csr_matrix
import numdifftools as nd
import scipy.optimize as optimize
import pickle

import random

# sympy 
from sympy import symbols,Symbol
import sympy as sym

# Quantum circuit
from qiskit import QuantumCircuit

# (Trotter) Exp pauli circuit : e^-iH
from qiskit.opflow import PauliTrotterEvolution

# parameters in q-circuits
from qiskit.circuit import ParameterVector

# Pauli matrices
from qiskit.opflow import I, X, Y, Z

# Qinfo package
import qiskit.quantum_info as qi

In [5]:

x, y ,z,U= symbols('x y z U')
theta=Symbol('theta',real=True)
phi=Symbol('phi',real=True)
lam=Symbol('lambda',real=True)


#Unitary transformation of 1 qubit
def Unitary(x,y,z):
    return np.matrix([[np.cos(x/2), np.exp(z*1j)*np.sin(x/2)], [-np.exp((y-z)*1j)*np.sin(x/2), np.exp(y*1j)*np.cos(x/2)]])

# Unitary transformation of 1 qubit
sym.Matrix([[sym.cos(x/2),sym.exp(z*sym.I)*sym.sin(x/2)],[-1*sym.exp((y-z)*sym.I)*sym.sin(x/2),sym.exp(y*sym.I)*sym.cos(x/2)]])


Matrix([
[                cos(x/2), exp(I*z)*sin(x/2)],
[-exp(I*(y - z))*sin(x/2), exp(I*y)*cos(x/2)]])

Qiskit one qubit gerneral rotation:

$U(\theta, \phi, \lambda) = \begin{pmatrix} \cos \left(\frac{\theta}{2}\right)  & -e^{i\lambda} \sin(
\frac{\theta}{2})\\ e^{i\phi} \sin(\frac{\theta}{2}) & e^{i(\lambda+\phi)}\cos(\frac{\theta}{2})  \end{pmatrix} $

$\theta= -x, \lambda= z, \phi= y-z  $

In [6]:
def QUnitary(x,y,z):
    circ=QuantumCircuit(1)
    circ.u(-1*x,y-z,z,0)
    
    # return quantum circuit and matrix form
    return circ , qi.Operator(circ)

##### Testing

In [5]:
(in1,in2,in3)=(random.random(),random.random(),random.random())

In [6]:
np.array(Unitary(in1,in2,in3)).round(8)==np.array(QUnitary(in1,in2,in3)[1]).round(8)

array([[ True,  True],
       [ True,  True]])

$H = \begin{pmatrix} a_1+a_2   & a_3+ i a_4\\ a_3-ia_4 & a_1-a_2  \end{pmatrix} $ =  $a_1 \times I + a_2 \times Z +a_3 \times X -a_4 \times Y$

In [7]:
#General Hamiltonian of 1 qubit
def Hamiltonian(a1,a2,a3,a4):
    return np.matrix([[a1+a2,a3+a4*1j], [a3-a4*1j, a1-a2]])

# Create general Hamiltonian H=a1*I+a2*Z+a3*X-a4*Y for all a_i belongs real, where I, X, Y, Z are Pauli matrices 
def QHamiltonian(a1,a2,a3,a4):
    return a1*I+a2*Z+a3*X-a4*Y 


# comment: change to a1*I+a2*X+a3*Y+a4*Z (?)

$(Q)H_{Free}=H_1 \otimes I + I \otimes H_2$

In [11]:
# Create general free Hamiltonian of 2 qubits = H \otimes 1 + 1 \otimes H 

#General free Hamiltonian of 2 qubits = H \otimes 1 + 1 \otimes H 
def Free(a1,a2,a3,a4,b1,b2,b3,b4):
    return np.kron(Hamiltonian(a1,a2,a3,a4),np.identity(2)) +  np.kron(np.identity(2),Hamiltonian(b1,b2,b3,b4))

# Tensor products are denoted with a caret : X^Y = X \otimes Y
def QFree(a1,a2,a3,a4,b1,b2,b3,b4):
    h1=QHamiltonian(a1,a2,a3,a4)
    h2=QHamiltonian(b1,b2,b3,b4)
    return (h1^I) + (I^ h2)

$H_{2Q}=\sum_{ij} g_{ij} P_i\otimes P_j$

In [8]:
#General interacting Hamiltonian of 2 qubits = \sum_ij g_ij Pauli_i\otimes Pauli_j

# for testing case define as global variable
Random= np.random.random((3, 3))*0.2  #g_ij

def Interacting():
    #Pauli matrices
    s1 = np.matrix([[0,1],[1,0]]) 
    s2 = np.matrix([[0,-1j],[1j,0]])
    s3 = np.matrix([[1,0],[0,-1]])
    Pauli= [s1,s2,s3]
    #Random= np.random.random((3, 3))*0.2  #g_ij
    A = np.asmatrix(np.zeros([4,4]))
    for i in range (3):
        for j in range(3):
             A = A + Random[i,j]*np.kron(Pauli[i],Pauli[j])
    return A

def QInteracting():
    #Random= np.random.random((3, 3))*0.2  #g_ij
    # initialize as 0*I^I (2-qubits) 
    A=0*I^I
    Pauli=[X,Y,Z]
    for i in range(3):
        for j in range(3):
            A+=Random[i,j]*(Pauli[i]^Pauli[j])
    return A

$e^{\sum_i c_i P_i \otimes P_i}$

In [9]:
#Exponential unitary = Exp[\sum_i c_i Pauli_i\otimes Pauli_i]
def Aux(c0,c1,c2):
    s1 = np.matrix([[0,1],[1,0]])
    s2 = np.matrix([[0,-1j],[1j,0]])
    s3 = np.matrix([[1,0],[0,-1]])
    Pauli= [s1,s2,s3]
    Matr = c0*np.kron(Pauli[0],Pauli[0]) + c1*np.kron(Pauli[1],Pauli[1]) + c2*np.kron(Pauli[2],Pauli[2]) 
    return np.asmatrix(expm(-1j*Matr))

def QAux(c0,c1,c2):
    Pauli=[X,Y,Z]
    Matr = (c0*(Pauli[0]^Pauli[0])+c1*(Pauli[1]^Pauli[1])+c2*(Pauli[2]^Pauli[2]))
    return Matr.exp_i()

In [12]:
#HAMILTONIAN (PARAMETERS AND CALCULATION)
p1 = 1
p2 = 2
p3 = 1
p4 = 3
q1 = 2
q2 = 1
q3 = 3
q4 = 5

Full = Free(p1,p2,p3,p4,q1,q2,q3,q4) + Interacting() #This computes the Hamiltonian
w, v = LA.eig(Full)

QFull = QFree(p1,p2,p3,p4,q1,q2,q3,q4) + QInteracting() #This computes the (Q)Hamiltonian
qw, qv = LA.eig(QFull.to_matrix())
print("Exact energies: ", w.real) #Eigenvalues
print("(Q)Exact energies: ", qw.real) #Eigenvalues
#print("Exact states: ", v.real) #Eigenvectors

Exact energies:  [12.73451153 -6.58972654  0.7157519   5.13946311]
(Q)Exact energies:  [12.73451153 -6.58972654  0.7157519   5.13946311]


In [13]:
#Full Unitary of 2 qubits = U_1 \otimes U_2 Exp[\sum_i c_i Pauli_i\otimes Pauli_i] U_3 \otimes U_4
def FullUnitary(x0, y0, z0, x1, y1, z1, x2, y2, z2, x3, y3,z3,c0,c1,c2):
    Auxhere = np.matmul(np.kron(Unitary(x0,y0,z0),Unitary(x1,y1,z1)),Aux(c0,c1,c2))
    return np.matmul(Auxhere,np.kron(Unitary(x2,y2,z2),Unitary(x3,y3,z3)))


In [14]:
def QFullUnitary(x0, y0, z0, x1, y1, z1, x2, y2, z2, x3, y3,z3,c0,c1,c2):
    # 2-qubits quantum circuit
    fullU_circ=QuantumCircuit(2)
    
    # quantum circuit for np.kron(Unitary(x0,y0,z0),Unitary(x1,y1,z1)) 
    # general unitary rotation on qubit '0' and general unitary rotation on qubit '1'  
    #### *****qiskit ordering***** 
    fullU_circ=fullU_circ.compose(QUnitary(x2,y2,z2)[0],[1])                                                                     
    fullU_circ=fullU_circ.compose(QUnitary(x3,y3,z3)[0],[0]) 
    
    # quantum circuit for exponential unitary = Exp[\sum_i c_i Pauli_i\otimes Pauli_i] 
    aux_circ=PauliTrotterEvolution(trotter_mode='trotter',reps=1).convert(QAux(c0,c1,c2)).to_circuit()
    fullU_circ=fullU_circ.compose(aux_circ)
                                                                          
    # quantum circuit for np.kron(Unitary(x2,y2,z2),Unitary(x3,y3,z3)) 
    # general unitary rotation on qubit 0, general unitary rotation on qubit 1                                     
    fullU_circ=fullU_circ.compose(QUnitary(x0,y0,z0)[0],[1])
    fullU_circ=fullU_circ.compose(QUnitary(x1,y1,z1)[0],[0])                                                                                 
    return fullU_circ

###### FullUnitary, QFullUnitary Testing

In [15]:
qin_test=tuple([random.random() for i in range(15)])
(in1,in2,in3,in4,in5,in6,in7,in8,in9,in10,in11,in12,in13,in14,in15)=qin_test

In [16]:
cf=FullUnitary(in1,in2,in3,in4,in5,in6,in7,in8,in9,in10,in11,in12,in13,in14,in15)

In [17]:
qf=qi.Operator(QFullUnitary(in1,in2,in3,in4,in5,in6,in7,in8,in9,in10,in11,in12,in13,in14,in15))

In [18]:
np.array(cf).round(8)==np.array(qf).round(8)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

#### Cost function

In [19]:
#Cost function of w-fields:
def cost(vec,FullH,w1,w2):
    v1 = np.matrix([1,0,0,0])
    v2 = np.matrix([0,1,0,0])
    v3 = np.matrix([0,0,1,0])
    v4 = np.matrix([0,0,0,1])
    Uni = FullUnitary(vec[0],vec[1],vec[2],vec[3],vec[4],vec[5],vec[6],vec[7],vec[8],vec[9], vec[10],vec[11],vec[12],vec[13],vec[14])
    cost1 = v1.dot(np.matmul(np.matmul(Uni.H,FullH),Uni)).dot(np.transpose(v1))
    cost2 = v2.dot(np.matmul(np.matmul(Uni.H,FullH),Uni)).dot(np.transpose(v2))
    cost3 = v3.dot(np.matmul(np.matmul(Uni.H,FullH),Uni)).dot(np.transpose(v3))
    cost4 = v4.dot(np.matmul(np.matmul(Uni.H,FullH),Uni)).dot(np.transpose(v4))
    LL = w1*w2*cost1+w1*(1-w2)*cost2+(1-w1)*w2*cost3+(1-w1)*(1-w2)*cost4
    return LL

def cost1(vec,FullH,w1,w2):
    v1 = np.matrix([1,0,0,0])
    v2 = np.matrix([0,1,0,0])
    v3 = np.matrix([0,0,1,0])
    v4 = np.matrix([0,0,0,1])
    Uni = FullUnitary(vec[0],vec[1],vec[2],vec[3],vec[4],vec[5],vec[6],vec[7],vec[8],vec[9], vec[10],vec[11],vec[12],vec[13],vec[14])
    cost1 = v1.dot(np.matmul(np.matmul(Uni.H,FullH),Uni)).dot(np.transpose(v1))
    return cost1

In [20]:
def Qcost(vec,FullH,w1,w2):
    # Define state from label (in computational basis)
    qv1=qi.Statevector.from_label('00')
    qv2=qi.Statevector.from_label('01')
    qv3=qi.Statevector.from_label('10')
    qv4=qi.Statevector.from_label('11')
    QUni = QFullUnitary(vec[0],vec[1],vec[2],vec[3],vec[4],vec[5],vec[6],vec[7],vec[8],vec[9], vec[10],vec[11],vec[12],vec[13],vec[14])
    
    # state after Unitary evolution:
    QUqv1=qv1.evolve(QUni)
    QUqv2=qv2.evolve(QUni)
    QUqv3=qv3.evolve(QUni)
    QUqv4=qv4.evolve(QUni)
    # expectation_value of given Hamiltonin:
    qcost1=QUqv1.expectation_value(FullH)
    qcost2=QUqv2.expectation_value(FullH)
    qcost3=QUqv3.expectation_value(FullH)
    qcost4=QUqv4.expectation_value(FullH)
    LL = w1*w2*qcost1+w1*(1-w2)*qcost2+(1-w1)*w2*qcost3+(1-w1)*(1-w2)*qcost4
    return LL

def Qcost1(vec,FullH,w1,w2):
    qv1=qi.Statevector.from_label('00')
    QUni = QFullUnitary(vec[0],vec[1],vec[2],vec[3],vec[4],vec[5],vec[6],vec[7],vec[8],vec[9], vec[10],vec[11],vec[12],vec[13],vec[14])
    qcost1 = qv1.evolve(QUni).expectation_value(FullH)
    return qcost1


##### Cost QCost Testing

In [21]:
# Classical
print(cost(qin_test,Full,0.7,0.9)[0,0].real)

# Quantum Circuit
print(Qcost(qin_test,QFull,0.7,0.9).real)
print("Difference: ", cost(qin_test,Full,0.7,0.9)[0,0].real-Qcost(qin_test,QFull,0.7,0.9).real)

4.859960934150066
4.8599609341500605
Difference:  5.329070518200751e-15


###### Cost1, QCost1 Testing

In [22]:
print(cost1(qin_test,Full,0.7,0.9)[0,0].real)
# Quantum Circuit
print(Qcost1(qin_test,QFull,0.7,0.9).real)
print("Difference: ", cost1(qin_test,Full,0.7,0.9)[0,0].real-Qcost1(qin_test,QFull,0.7,0.9).real)

6.774006972000681
6.774006972000673
Difference:  7.993605777301127e-15


### Energies

In [23]:
#Calculations of the Energies with a given set of parameters
def energies(vec,FullH):
    v1 = np.matrix([1,0,0,0])
    v2 = np.matrix([0,1,0,0])
    v3 = np.matrix([0,0,1,0])
    v4 = np.matrix([0,0,0,1])
    Uni = FullUnitary(vec[0],vec[1],vec[2],vec[3],vec[4],vec[5],vec[6],vec[7],vec[8],vec[9], vec[10],vec[11],vec[12],vec[13],vec[14])
    cost1 = v1.dot(np.matmul(np.matmul(Uni.H,FullH),Uni)).dot(np.transpose(v1))
    cost2 = v2.dot(np.matmul(np.matmul(Uni.H,FullH),Uni)).dot(np.transpose(v2))
    cost3 = v3.dot(np.matmul(np.matmul(Uni.H,FullH),Uni)).dot(np.transpose(v3))
    cost4 = v4.dot(np.matmul(np.matmul(Uni.H,FullH),Uni)).dot(np.transpose(v4))
    E1n = np.asarray(cost1)[0][0].real
    E2n = np.asarray(cost2)[0][0].real
    E3n = np.asarray(cost3)[0][0].real
    E4n = np.asarray(cost4)[0][0].real
    return [E1n,E2n,E3n,E4n]

In [24]:
def Qenergies(vec,FullH):
    qv1=qi.Statevector.from_label('00')
    qv2=qi.Statevector.from_label('01')
    qv3=qi.Statevector.from_label('10')
    qv4=qi.Statevector.from_label('11')
    QUni = QFullUnitary(vec[0],vec[1],vec[2],vec[3],vec[4],vec[5],vec[6],vec[7],vec[8],vec[9], vec[10],vec[11],vec[12],vec[13],vec[14])
    # state after Unitary evolution:
    QUqv1=qv1.evolve(QUni)
    QUqv2=qv2.evolve(QUni)
    QUqv3=qv3.evolve(QUni)
    QUqv4=qv4.evolve(QUni)
    # expectation_value of given Hamiltonin:
    qcost1=QUqv1.expectation_value(FullH).real
    qcost2=QUqv2.expectation_value(FullH).real
    qcost3=QUqv3.expectation_value(FullH).real
    qcost4=QUqv4.expectation_value(FullH).real
    return [qcost1,qcost2,qcost3,qcost4]

##### Testing

In [25]:
print(energies(qin_test,Full))
print(Qenergies(qin_test,QFull))
d=np.array(energies(qin_test,Full))-np.array(Qenergies(qin_test,QFull))
print("Difference: "+'\n',d)

[6.774006972000681, 1.799099344590923, 1.5149699048584175, 1.9119237785499825]
[6.774006972000673, 1.7990993445909202, 1.5149699048584149, 1.9119237785499819]
Difference: 
 [7.99360578e-15 2.66453526e-15 2.66453526e-15 6.66133815e-16]


(TO DO)

In [25]:
class functioncost():
    
    def __init__(self, Hamilt,w1,w2):
        self.Hamilt = Hamilt
        self.w1 = w1
        self.w2 = w2
    def evalua(self,seed):
        return cost(seed,self.Hamilt,self.w1,self.w2).real

class functioncost1():
    def __init__(self, Hamilt,w1,w2):
        self.Hamilt = Hamilt
        self.w1 = w1
        self.w2 = w2
    def evalua(self,seed):
        return cost1(seed,self.Hamilt,self.w1,self.w2).real    